In [41]:
from gurobipy import *
import numpy as np
import math as mt

In [42]:
# Poser le nombre des varialbles et des contraintes
nbcont=5
nbvar=11
w=[2,1]
n=2
p1=3
p=5
print(w)

[2, 1]


In [43]:
# Creer une matrice utilite
u=np.zeros(5)
u1= np.array([5,6,4,8,1])
u=np.vstack((u1,u))
u=np.delete(u, (n-1), axis=0)
u1= np.array([3,8,6,2,5])
u=np.vstack((u,u1))
print(u)

[[5. 6. 4. 8. 1.]
 [3. 8. 6. 2. 5.]]


In [44]:
# Creer le vecteur w'
w1=[]
for i in range(n):
    if i==n-1 :
        w1.append(w[i])
    else :
        w1.append(w[i]-w[i+1])

print(w1)

[1, 1]


In [45]:
# 2) generer la matrice des contraintes des x

L=np.array([])
L2=np.zeros((p,1))
b=np.zeros(p)

u_prim=-1*u


for k in range(n):
     for i in range (n):
          for j in range(p):
               L=np.append(L,[u_prim[k,j]])
   
          b=np.vstack((b,L))  
          L=np.array([])
b=np.delete(b, [0], axis=0)

L=np.ones(p)
b=np.vstack((b,L)) 



# 2) generer la matrice des contraintes des bik
L=np.identity(n*n)
L=-1*L
for i in range (nbcont-n*n):
     L1=np.zeros(n*n)
     L =np.vstack((L,L1))


# 2) concatener la matrice des contraintes des bik et des xj dans la matrice b

b=np.concatenate((L,b),axis=1)


# 3) generer la matrice des contraintes des rk
L2=np.zeros((1,n))
for i in range(n):
    L1=np.identity(n)
    L2=np.vstack((L1,L2))


# 2) concatener la matrice des contraintes rk et b
b=np.concatenate((L2,b),axis=1)
print(b)

[[ 1.  0. -1. -0. -0. -0. -5. -6. -4. -8. -1.]
 [ 0.  1. -0. -1. -0. -0. -5. -6. -4. -8. -1.]
 [ 1.  0. -0. -0. -1. -0. -3. -8. -6. -2. -5.]
 [ 0.  1. -0. -0. -0. -1. -3. -8. -6. -2. -5.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.]]


In [46]:
# Second membre
S= [0 for i in range (n*n)]
S.append(3)

print(S)

[0, 0, 0, 0, 3]


In [47]:
# Coefficients de la fonction objectif
# generer les Coefficients des r
a=[]
for i in range(1,n+1):
    a.append(i*w1[i-1])
# generer les Coefficients des bik
for i in range(n):
    for j in range(n):
        a.append(-1*w1[j])
# generer les Coefficients des x
for i in range (p):
    a.append(0)
print(a)

[1, 2, -1, -1, -1, -1, 0, 0, 0, 0, 0]


In [48]:
# Creer les models de la fonction objectif
m = Model("exercice1.4") 

In [49]:
# declaration des variables de decisions
x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((i+1,(j+1)))))
for j in range(p):
    x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))

# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var r1>, <gurobi.Var r2>, <gurobi.Var b11>, <gurobi.Var b12>, <gurobi.Var b21>, <gurobi.Var b22>, <gurobi.Var x1>, <gurobi.Var x2>, <gurobi.Var x3>, <gurobi.Var x4>, <gurobi.Var x5>]


In [50]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(nbvar):
    obj += a[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [51]:
# Definition des contraintes
lignes = range(nbcont)
colonnes = range(nbvar)

for i in lignes[:-1]:
    m.addConstr(quicksum(b[i][j]*x[j] for j in colonnes) <= S[i], "Contrainte%d" % i)
    
m.addConstr(quicksum(b[-1][j]*x[j] for j in colonnes) == S[-1], "Contrainte%d" % i)

<gurobi.Constr *Awaiting Model Update*>

In [52]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 11 columns and 33 nonzeros
Model fingerprint: 0xd87a7057
Variable types: 6 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 45.0000000
Presolve time: 0.00s
Presolved: 5 rows, 11 columns, 29 nonzeros
Variable types: 6 continuous, 5 integer (5 binary)

Root relaxation: objective 5.000000e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      50.0000000   50.00000  0.00%     -    0s

Explored 1 nodes (7 simplex itera